In [ ]:
from requests import exceptions
import requests
import cv2
import os
import gevent

In [ ]:
searchengine = 'c3po'
output = 'download/c3po1'

In [ ]:
API_KEY = "Input your key"
MAX_RESULTS = 50
GROUP_SIZE = 250

In [ ]:
URL = "https://api.bing.microsoft.com/v7.0/images/search"
       


EXCEPTIONS = {IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError, exceptions.ConnectionError,
              exceptions.Timeout}

In [ ]:
term = searchengine
headers = {"Ocp-Apim-Subscription-Key": API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}

In [ ]:
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

In [ ]:
total = 0
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults, term))

In [ ]:
def grab_page(url, ext, total):

    try:
        print("[INFO] fetching: {}".format(url))
        r = requests.get(url, timeout=30)
        p = os.path.sep.join([output, "{}{}".format(
            str(total).zfill(8), ext)])

        f = open(p, "wb")
        f.write(r.content)
        f.close()

        image = cv2.imread(p)

        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            return

    except Exception as e:
        if type(e) in EXCEPTIONS:
            print("[INFO] skipping: {}".format(url))
            return

In [ ]:
for offset in range(0, estNumResults, GROUP_SIZE):
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    jobs = []
    for v in results["value"]:
        total += 1
        ext = v["contentUrl"][v["contentUrl"].rfind("."):]
        url = v["contentUrl"]
        
        jobs.append(gevent.spawn(grab_page, url, ext, total))

    gevent.joinall(jobs, timeout=10)
    print(total)